In [41]:
import pandas as pd
import numpy as np

Read Data 

In [42]:
# mktdata = pd.ExcelFile("https://github.com/Rajwantmishra/msds/blob/master/612/DataInContext/data/QT_Data-TEST.xlsx")

# Note two method used here to read excel . One m

mktdata_train = pd.ExcelFile("C:/Users/rajwa/Documents/GitHub/msds/612/DataInContext/data/QT_DataTRAIN.xlsx")
mktdata_test = pd.read_excel("C:/Users/rajwa/Documents/GitHub/msds/612/DataInContext/data/QT_Data-TEST.xlsx",)

mktdata = pd.read_excel("C:/Users/rajwa/Documents/GitHub/msds/612/DataInContext/data/Q1.xlsx",)

Train_MKT = mktdata_train.parse()
Test_MKT = mktdata_test 

In [43]:
# Head data 
mktdata.head()

,KUNNR_NEW,TYPE.x,Brand,External Description,city,state,Order Quantity
0,1254314,PREMIUM FASHION BRANDS,HC,LUX12,NORTH BELLMORE,NY,18
1,1263875,RAY-BAN BRANDS,RX,LUX12,NEW BRAUNFELS,TX,15
2,1263875,RAY-BAN BRANDS,RY,LUX12,NEW BRAUNFELS,TX,4
3,1267631,RAY-BAN BRANDS,RX,LUX12,ROSEBURG,OR,15
4,1267631,RAY-BAN BRANDS,RY,LUX12,ROSEBURG,OR,4


In [44]:
# tail some data 
mktdata.tail()

,KUNNR_NEW,TYPE.x,Brand,External Description,city,state,Order Quantity
62807,1222228,OAKLEY BRANDS,OX,NB8,LOS ANGELES,CA,17
62808,1222228,OAKLEY BRANDS,OY,NB8,LOS ANGELES,CA,2
62809,1267974,OAKLEY BRANDS,OX,CODE F,CLOVIS,CA,3
62810,1274276,RAY-BAN BRANDS,RB,NB8,BRIDGEPORT,TX,9
62811,1274276,RAY-BAN BRANDS,RX,NB8,BRIDGEPORT,TX,16


In [47]:
mktdata.columns

Index(['KUNNR_NEW', 'TYPE.x', 'Brand', 'External Description', 'city', 'state',
       'Order Quantity'],
      dtype='object')

In [89]:
lean_mktdata = mktdata[['KUNNR_NEW','Brand','Order Quantity']]
lean_mktdata.dtypes


KUNNR_NEW          int64
Brand             object
Order Quantity     int64
dtype: object

In [90]:
# Converting Numeric to object 
# https://stackoverflow.com/questions/15891038/change-data-type-of-columns-in-pandas

lean_mktdata =  lean_mktdata.astype({'KUNNR_NEW':'object'})
lean_mktdata.dtypes

KUNNR_NEW         object
Brand             object
Order Quantity     int64
dtype: object

In [91]:
lean_mktdata.describe()

,Order Quantity
count,62812.000000
mean,14.474877
std,24.450265
min,1.000000
25%,6.000000
50%,10.000000
75%,18.000000
max,2248.000000


In [72]:
lean_mktdata.dtypes

KUNNR_NEW         object
Brand             object
Order Quantity     int64
dtype: object

In [93]:
lean_mktdata.dtypes
#$ lean_mktdata['KUNNR_NEW'] = lean_mktdata['KUNNR_NEW'].astype(object,errors='ignore' )


KUNNR_NEW         object
Brand             object
Order Quantity     int64
dtype: object

In [94]:
# Check Train Dataset
Train_MKT.head()

,Row Labels,FAST FASHION BRANDS,LUXURY BRANDS,OAKLEY BRANDS,PREMIUM FASHION BRANDS,PROFESSIONAL BRANDS,RAY-BAN BRANDS
0,1222060,NaN,32.0,NaN,NaN,NaN,NaN
1,1222061,NaN,119.0,121.0,66.0,NaN,67.0
2,1222062,17.0,NaN,NaN,18.0,NaN,NaN
3,1222064,NaN,NaN,12.0,NaN,NaN,NaN
4,1222065,NaN,NaN,20.0,31.0,NaN,22.0


In [96]:
# Check Test Dataset 
lean_mktdata.head()

,KUNNR_NEW,Brand,Order Quantity
0,1254314,HC,18
1,1263875,RX,15
2,1263875,RY,4
3,1267631,RX,15
4,1267631,RY,4


# Recommendation my Count 
So when a customer places order one brand from our website and has bought a product, we want to recommend them other Brand that similar people bought. So in this exercise, we’re going to use data about which customers bought which Brand and, based on that, build an Category to Category affinity score, and then use it for recommendation.

In [99]:
df_matrix = pd.pivot_table(lean_mktdata, values='Order Quantity', index='KUNNR_NEW', columns='Brand')
df_matrix.head()

Brand,AR,AX,BB,BE,BV,CH,DG,DY,EA,HC,...,RJ,RL,RX,RY,SF,TF,TY,VA,VE,VO
KUNNR_NEW,,,,,,,,,,,,,,,,,,,,,
1221894,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,...,NaN,NaN,10.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1221920,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1221922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,...,NaN,NaN,27.0,6.0,NaN,NaN,20.0,NaN,15.0,NaN
1221923,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,15.0,...,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
# calcualting Affinity Matrix
# Get list of unique items
catList=list(set(lean_mktdata["Brand"].tolist()))
catList

['RA',
 'OY',
 'PR',
 'RY',
 'BV',
 'EA',
 'PP',
 'TF',
 'CH',
 'VO',
 'AR',
 'MU',
 'DY',
 'AX',
 'PO',
 'RB',
 'DG',
 'PH',
 'MK',
 'SF',
 'BE',
 'PS',
 'HC',
 'BB',
 'OX',
 'VE',
 'OJ',
 'RL',
 'VA',
 'OO',
 'RJ',
 'RX',
 'TY']

In [120]:
#Get count of Customers
custCount=len(set(lean_mktdata["KUNNR_NEW"].tolist()))
#Create an empty data frame to store item affinity scores for items.
catAffinity= pd.DataFrame(columns=('Brand1', 'Brand2', 'score'))
rowCount=0
#For each category/brand in the list, compare with other brand.
for ind1 in range(len(catList)):
    
    #Get list of Customer who bought this Category 1.
    item1Users = lean_mktdata[lean_mktdata['Brand']==catList[ind1]]["KUNNR_NEW"].tolist()
    #print("Item 1 ", item1Users)
    
    #Get item 2 - items that are not item 1 or those that are not analyzed already.
    for ind2 in range(ind1, len(catList)):
        
        if ( ind1 == ind2):
            continue
       
        #Get list of users who bought item 2
        item2Users=lean_mktdata[lean_mktdata['Brand']==catList[ind2]]["KUNNR_NEW"].tolist()
        #print("Item 2",item2Users)
        
        #Find score. Find the common list of users and divide it by the total users.
        commonUsers= len(set(item1Users).intersection(set(item2Users)))
        score=commonUsers / custCount

        #Add a score for item 1, item 2
        catAffinity.loc[rowCount] = [catList[ind1],catList[ind2],score]
        rowCount +=1
        #Add a score for item2, item 1. The same score would apply irrespective of the sequence.
        catAffinity.loc[rowCount] = [catList[ind2],catList[ind1],score]
        rowCount +=1
        
#Check final result
catAffinity.head()


,Brand1,Brand2,score
0,RA,OY,0.005567
1,OY,RA,0.005567
2,RA,PR,0.005196
3,PR,RA,0.005196
4,RA,RY,0.008166


We are going to take every item and find the affinity of that item to other items, and the way I’m going to do it is by finding out how many customers have bought both these products. The more number of customers who buy both the products, the higher is going to be the affinity score. So that is how the affinity score I am going to be computing here, and after that you can see that I am printing out the affinity score between each items. Item one to item two has a high affinity score of .4, whereas 5,001 to 5,003, there is no affinity at all.

In [123]:
catAffinity.sort_values(by=['score'], ascending=False).head()

,Brand1,Brand2,score
781,RX,RB,0.530018
780,RB,RX,0.530018
992,OX,OO,0.294609
993,OO,OX,0.294609
241,RX,RY,0.270298


So this is the list that come up with the list of affinity scores. So how do we do recommendations? Let us say the customer bought a product 5,001. We want to find what items want to recommend to him. And for that, We are going to go back to this table, go to all the records that are item one in the first column, and get the list of all the items two and their scores. And we can do that in descending order. And those items that you see here is what I want to recommend.

In [156]:
searchBrand='RB'
recoList=catAffinity[catAffinity.Brand1==searchBrand]\
        [["Brand2","score"]]\
        .sort_values("score", ascending=[0])
        
print("Recommendations for item 5001\n", recoList)


Recommendations for item 5001
     Brand2     score
780     RX  0.530018
209     RY  0.251461
762     HC  0.179363
766     OX  0.175466
776     OO  0.164703
754     MK  0.130463
768     VE  0.102069
91      OY  0.100863
782     TY  0.098079
151     PR  0.097615
752     PH  0.076088
750     DG  0.069036
758     BE  0.066252
515     VO  0.065232
421     TF  0.061984
778     RJ  0.057716
319     EA  0.054189
772     RL  0.039065
679     AX  0.035539
760     PS  0.030621
559     AR  0.026445
715     PO  0.023105
770     OJ  0.019393
265     BV  0.016888
641     DY  0.016331
29      RA  0.015310
764     BB  0.015218
469     CH  0.013083
601     MU  0.012898
371     PP  0.012712
774     VA  0.011413
756     SF  0.009836


In [236]:
searchBrand='CH'
recoList=catAffinity[catAffinity.Brand1==searchBrand]\
        [["Brand2","score"]]\
        .sort_values("score", ascending=[0])
        
print("Recommendations for item 5001\n", recoList.head())


Recommendations for item 5001
     Brand2     score
468     RB  0.013083
500     RX  0.011599
137     PR  0.007980
486     OX  0.007423
496     OO  0.006866


    How can we Identify the Customer that can be prospects for New Category ?

In [140]:
# Find Customer with Brand 'CH' and all the other Brands they are open for
df_matrix_CH = pd.pivot_table(lean_mktdata[lean_mktdata['Brand']=='CH'], values='Order Quantity', index='KUNNR_NEW', columns='Brand')
df_matrix_CH.head()

Brand,CH
KUNNR_NEW,
1221961,19.00
1221972,12.25
1221976,12.00
1222032,8.00
1230907,40.00


In [237]:
Ext_CH_Cust = lean_mktdata[lean_mktdata['Brand']=='CH']['KUNNR_NEW']
Ext_CH_Brand = recoList.head(5)['Brand2']


'CH' brand customers are stored in Ext_CH_Cust and CH possible top 5 brands are stored in Ext_CH_Brand

In [238]:
Ext_CH_Cust.head() 
Ext_CH_Brand.head()
Ext_CH_Brand

468    RB
500    RX
137    PR
486    OX
496    OO
Name: Brand2, dtype: object

Ext_CH_Cust

In [163]:
Ext_CH_Cust_list = pd.merge(Ext_CH_Cust,lean_mktdata,on=['KUNNR_NEW'],how='left')
Ext_CH_Cust.head()
lean_mktdata.head()

,KUNNR_NEW,Brand,Order Quantity
0,1254314,HC,18
1,1263875,RX,15
2,1263875,RY,4
3,1267631,RX,15
4,1267631,RY,4


In [192]:
Ext_CH_Cust_list = Ext_CH_Cust_list[['KUNNR_NEW','Brand']].sort_values(by=['KUNNR_NEW'])

Ext_CH_Cust_list = Ext_CH_Cust_list.drop_duplicates()
Ext_CH_Cust_list['Value'] = 1
Ext_CH_Cust_list.head()

,KUNNR_NEW,Brand,Value
6293,1221961,CH,1
403,1221972,TF,1
397,1221972,BE,1
3980,1221972,CH,1
399,1221972,MK,1


In [239]:
# build the list of brand top 10
top10 = recoList.head(10)['Brand2'].tolist()
top10
# recoList.head(5)['Brand2'].tolist()

['RB', 'RX', 'PR', 'OX', 'OO', 'TF', 'RY', 'PO', 'PS', 'OY']

In [240]:
# Find Customer with Brand 'CH' and all the other Brands they are open for

df_matrix_CH = pd.pivot_table(Ext_CH_Cust_list[Ext_CH_Cust_list.Brand.isin(top10)], values='Value', index='KUNNR_NEW', columns='Brand')
df_matrix_CH

Brand,OO,OX,OY,PO,PR,PS,RB,RX,RY,TF
KUNNR_NEW,,,,,,,,,,
1221972,1.0,1.0,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0
1221976,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
1222032,1.0,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,1.0
1230907,1.0,NaN,NaN,1.0,1.0,1.0,1.0,NaN,NaN,1.0
1231250,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1246409,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,1.0
1246528,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
1246650,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,NaN,1.0
1246867,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
